Для начала заходим на https://dumps.wikimedia.org/ruwiki/20210501/ и скачиваем ruwiki-20210501-pages-articles-multistream-index.txt или более позднюю версию если она доступна, в этом файле лежит список всех актуальных русскоязычных статей википедии

Подключаем все необходимые библиотеки, у меня стоит Python 3.9.1, все библиотеки скачиваются просто через pip install, но думаю у вас они и так есть  ;)

In [215]:
import pandas as pd
import wikipediaapi
import tree
from tqdm import tqdm, trange
import time
from multiprocessing import Pool
import re

Прочитаем скачанный ранее файл в dataframe pandas

In [216]:
russian_articles_df = pd.read_csv('ruwiki-20210501-pages-articles-multistream-index.txt', delimiter='\t', header=None)
russian_articles = list(russian_articles_df[0])

russian_articles_df.head(5)

,0
0,780:4:Базовая статья
1,780:7:Литва
2,780:9:Россия
3,780:10:Слоновые
4,780:11:Мамонты


In [217]:
russian_articles[0:5]

['780:4:Базовая статья',
 '780:7:Литва',
 '780:9:Россия',
 '780:10:Слоновые',
 '780:11:Мамонты']

Уберем лишнюю часть в начале и оставим только сами названия статей

In [218]:
russian_articles = list(map(lambda elem: ' '.join(elem.split(':')[2:]), russian_articles))

In [219]:
russian_articles[0:5]

['Базовая статья', 'Литва', 'Россия', 'Слоновые', 'Мамонты']

Добавим названия статей в наш dataframe

In [220]:
russian_articles_df['article'] = russian_articles
russian_articles_df.head(5)

,0,article
0,780:4:Базовая статья,Базовая статья
1,780:7:Литва,Литва
2,780:9:Россия,Россия
3,780:10:Слоновые,Слоновые
4,780:11:Мамонты,Мамонты


Посмотрим, сколько всего статей, есть в списке

In [221]:
len(russian_articles_df)

4870048

Воспользуемся wikipediaapi для того, чтобы получать доступ к статьям по их названию

Документация по wikipediaapi - [здесь](https://pypi.org/project/Wikipedia-API/)

In [222]:
wiki_wiki = wikipediaapi.Wikipedia(
        language='ru',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

Прежде чем обходить все русскоязычную википедию, сначала рассмотрим пример

In [223]:
p_wiki = wiki_wiki.page(russian_articles_df['article'][0])
print(p_wiki.text)

Первая 'Базовая статья' нам не подходит, она пустая, посмотрим вторую

In [224]:
p_wiki = wiki_wiki.page(russian_articles_df['article'][1])
print(p_wiki.text)

Литва́ (лит. Lietuva [lʲɪɛtʊˈvɐ]), официальное название — Лито́вская Респу́блика (лит. Lietuvos Respublika) — государство, расположенное в северной части Европы.
Площадь — 65 300 км². Протяжённость с севера на юг — 280 км, а с запада на восток — 370 км. Население составляет 2 795 175 человек (январь, 2021). Занимает 137-е место в мире по численности населения и 121-е по территории. Имеет выход к Балтийскому морю, расположена на его восточном побережье. Береговая линия составляет всего 99 км (наименьший показатель среди государств Балтии). На севере граничит с Латвией, на юго-востоке — с Белоруссией, на юго-западе — с Польшей и Калининградской областью России. По площади и населению является самым крупным прибалтийским государством.
Сто­ли­ца — Вильнюс. Официальный язык — ли­тов­ский. Де­неж­ная еди­ни­ца — евро.
Восстановление независимости страны провозглашено 11 марта 1990 года,. 6 сентября 1991 года Государственный совет СССР признал независимость Литвы. 
Литва — член ООН (1991), ОБ

In [ ]:
processed_p_wiki = re.sub(r'[^а-я -]', '', p_wiki.text.lower())
print(processed_p_wiki)

In [ ]:
processed_p_wiki = re.findall(r'\b[А-я-]+\b', p_wiki.text.lower())
print(processed_p_wiki)

Оставим в статье только слова состоящие из кирилицы

In [225]:
def remove_special_chars(text,char_list):
    for char in char_list:
        text=text.replace(char,' ')
    return text.replace(u'\xa0', u' ') 

chars = ['\n']
processed_p_wiki = re.sub(r'[^а-я -]', '', remove_special_chars(p_wiki.text.lower(), chars))
print(processed_p_wiki)

литва лит   официальное название  литовская республика лит    государство расположенное в северной части европы площадь    км протяжнность с севера на юг   км а с запада на восток   км население составляет    человек январь  занимает -е место в мире по численности населения и -е по территории имеет выход к балтийскому морю расположена на его восточном побережье береговая линия составляет всего  км наименьший показатель среди государств балтии на севере граничит с латвией на юго-востоке  с белоруссией на юго-западе  с польшей и калининградской областью россии по площади и населению является самым крупным прибалтийским государством столица  вильнюс официальный язык  литовский денежная единица  евро восстановление независимости страны провозглашено  марта  года  сентября  года государственный совет ссср признал независимость литвы  литва  член оон  обсе  совета европы  вто  европейского союза  нато  и оэср  входит в шенгенскую зону и еврозону  этимология этимология слова литва точно не из

Сделаем из статьи список слов и удалим из него все пустые строки

In [226]:
processed_text = list(filter(None, processed_p_wiki.split(' ')))
processed_text[:10]

['литва',
 'лит',
 'официальное',
 'название',
 'литовская',
 'республика',
 'лит',
 'государство',
 'расположенное',
 'в']

Используем дерево, чтобы сохранять в него слова

In [231]:
tree = tree.Trie()

In [ ]:
for word in processed_text:
    tree.add(word)

Используем функцию sentence, чтобы проверять, есть ли слова в предложении в дереве, заранее предпологаем, что нас интересуют только слова написанные кирилицей

In [232]:
def sentence(string):
    ans = True
    for word in list(filter(None, re.sub(r'[^а-я -]', '', remove_special_chars(string, chars)).split(' '))):
        #print(tree.contains(word))
        ans = ans and tree.contains(word)
    return ans

In [ ]:
sentence('декабря взвешивает возможность создать литовское королевство с германским монархом  марта')

In [ ]:
sentence('Писать можно на любом языке программирования, но лучше')

Используем функцию process_full_article чтобы скачать статью, обработать и записать все слова в дерево; если с первого раза не получается скачать статью, то функция будет пытаться сделать это еще n_attempts раз, с временным промежутком t_sleep

In [233]:
def process_full_article(article, n_attempts=3, t_sleep=1):
    for _ in range(n_attempts):
        try:
            current = wiki_wiki.page(article).text
            chars = ['\n']
            processed_article = re.sub(r'[^а-я -]', '', remove_special_chars(current.lower(), chars))
            processed_text = list(filter(None, processed_article.split(' ')))
            for word in processed_text:
                tree.add(word)
            break
        except Exception as exc:
            time.sleep(t_sleep)

Посмотрим на первой статье:

In [234]:
process_full_article(russian_articles_df['article'][1])

Проверим слова которые есть в статье:

In [235]:
sentence('декабря взвешивает возможность создать литовское королевство с германским монархом  марта')

True

Теперь возьмем предложение из вообще другой статьи:

In [236]:
sentence('Писать можно на любом языке программирования, но лучше')

False

Я сначала честно и добросовестно пытался обойти все статьи из списка, но даже с учетом multiprocessing это заняло всю ночь и так и не закончило работать 

In [ ]:
p = Pool()
p.map(process_full_article, russian_articles_df['article'])
p.close()
p.join()

Поэтому в качетсве примера я решил взять выборку из первой тысячи статей:

In [237]:
for article in tqdm(russian_articles_df['article'][:1000]):
    process_full_article(article)

100%|██████████| 1000/1000 [28:30<00:00,  1.71s/it]


В чем недостатки этого способа построения спеллчекера?

- Википедия использует в основном научный стиль речи, из-за чего многие слова, которые например используются в разговорной речи могут просто не встретиться, несмотря на огромное кол-во статей 

- В википедии много устаревших слов, которые уже не актуальны и не используются

Быстро ли работает ваш спеллчекер? Можно ли его ускорить и если да, то как?

Я постарался максимально ускорить спеллчекер, добавляя слова в дерево, а не в обычный словарь или set()

По моим оценкам такой спеллчекер должен работать за log(N), где основание у логарифма число букв в алфавите

На скорость работы спеллчекера посмотрим ниже через %%time

Приведите интересные примеры правильных слов, которые этот спеллчекер считает ошибочными и наоборот, слов с опечаткой, которые считаются им правильными.

В википедии мало используется речь от первого лица, поэтому многие формы глаголов там ни разу не встретятся

In [238]:
%%time
sentence('я танцую, работаю, играю')

CPU times: user 339 µs, sys: 5 µs, total: 344 µs
Wall time: 347 µs


False

Слова, которые не имеют для русского человека никакого смысла вне научного контеста или попросту устарели, но тем не менее там есть

In [240]:
%%time
sentence('шнеками зипунами летава миндовга')

CPU times: user 46 µs, sys: 1 µs, total: 47 µs
Wall time: 53.2 µs


True

Аналогичное решение, которое я мог бы преддложить - это использование другого [wikipediaapi](https://pypi.org/project/wikipedia/) у которого есть метод search, так мы могли бы пробегать слова в предложении искать релевантные статьи и в них уже искать нужные нам слова, это аpi устарел и если для того, чтобы пользоваться search он подходит, то для самих статей я бы использовал первый [api](https://pypi.org/project/Wikipedia-API/) 

[Пример работы подобным образом](https://youtu.be/yhbHM0Fzo-U)